In [17]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import requests
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry
import json
import time



In [19]:
df = pd.read_csv('output.csv')
df['is_win'] = df['is_win'].astype(int)

hero_id = pd.read_csv('hero_info.csv')

# grab data for win and loss
#df_win = df[df['is_win'] == 1]
#df_lose = df[df['is_win'] == 0]
df.head()

,match_uid,player_uid,name,hero_id,is_win,kills,deaths,assists,hero_damage,hero_healed,damage_taken,heroes
0,6714034_1739047847_94_11001_10,1760979252,Boogerscrotum,1050,0,2,4,3,1434.000000,6792.121636,5171.407166,"[{'hero_id': 1050, 'playtime': {'minutes': 7, ..."
1,6714034_1739047847_94_11001_10,2060844458,SpiderManBulge,1023,0,3,8,3,6320.204075,2842.749824,11076.655098,"[{'hero_id': 1040, 'playtime': {'seconds': 0, ..."
2,6714034_1739047847_94_11001_10,946297425,WoozyMckay,1041,0,5,6,5,7897.712778,6203.631268,3900.479958,"[{'hero_id': 1031, 'playtime': {'minutes': 2, ..."
3,6714034_1739047847_94_11001_10,294125044,Portal112,1029,0,10,8,0,5170.035370,0.000000,4896.128411,"[{'hero_id': 1029, 'playtime': {'minutes': 7, ..."
4,6714034_1739047847_94_11001_10,1816976623,DraconianSpy,1018,0,4,6,0,6029.966664,0.000000,19273.281858,"[{'hero_id': 1011, 'playtime': {'minutes': 5, ..."


In [29]:
params = ['id', 'name', 'attack_type', 'role']
hero_info_df = hero_id[params]
hero_info_df.head()


,id,name,attack_type,role
0,1011,Bruce Banner,Melee Heroes,VANGUARD
1,1014,The Punisher,Hitscan Heroes,DUELIST
2,1015,Storm,Projectile Heroes,DUELIST
3,1016,Loki,Projectile Heroes,STRATEGIST
4,1018,Doctor Strange,Projectile Heroes,VANGUARD


***
## Purpose

The goal of this file is to perform the same analysis as the first file but classify the data further into their respective roles.


In [ ]:
import ast


data_unpacked = pd.DataFrame()

player_heros = df[['match_uid','player_uid','heroes']]

for row_num, data in enumerate(player_heros['heroes']):
    # Parse the string using ast.literal_eval
    row_data = ast.literal_eval(data)

    # Create a DataFrame from the list of dictionaries
    row_df = pd.DataFrame(row_data)

    data_unpacked = pd.concat([data_unpacked, row_df], ignore_index=True)

display(data_unpacked)

,hero_id,playtime,kills,deaths,assists,hit_rate
0,1050,"{'minutes': 7, 'seconds': 39, 'raw': 459}",2,4,3,0.547771
1,1040,"{'seconds': 0, 'raw': 0}",0,0,0,0.000000
2,1039,"{'minutes': 2, 'seconds': 33, 'raw': 153}",0,3,1,0.475000
3,1041,"{'minutes': 1, 'seconds': 42, 'raw': 102}",3,2,0,0.540541
4,1023,"{'minutes': 3, 'seconds': 35, 'raw': 215}",0,3,2,0.325648
...,...,...,...,...,...,...
452,1025,"{'minutes': 3, 'seconds': 44, 'raw': 224}",2,4,1,0.927536
453,1031,"{'minutes': 12, 'seconds': 23, 'raw': 743}",19,9,8,0.492338
454,1038,"{'minutes': 3, 'seconds': 33, 'raw': 213}",4,3,0,0.512048
455,1047,"{'minutes': 12, 'seconds': 12, 'raw': 732}",3,6,28,0.641026
